###DataFrame

A DataFrame is equivalent to a relational table in Spark SQL, and can be created using various functions in SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("joins").getOrCreate()

In [3]:
from pyspark.sql.types import IntegerType,StructField,StructType,DoubleType,LongType,DecimalType

In [4]:
data_schema = [StructField("id_adesao", DecimalType(30,0), nullable=False)
               ,StructField("id_cancelado", DecimalType(30,0), nullable=True)
               ,StructField("id_vendedor", LongType(), nullable=True)
               ,StructField("id_portal", IntegerType(), nullable=True)
               ,StructField("id_loja", DecimalType(30,0), nullable=True)
               ,StructField("id_dt_emissao", IntegerType(), nullable=True)
               ,StructField("id_dt_canc", IntegerType(), nullable=True)
               ,StructField("valor_premio", DoubleType(), nullable=False)
               ,StructField("valor_produto", DoubleType(), nullable=False)]

final_struc = StructType(fields=data_schema)

In [5]:
fato = spark.read.csv('/FileStore/tables/fato_seguro.csv',schema=final_struc,header=True)

fato.show(5)

+--------------------+--------------------+------------+---------+--------------------+-------------+----------+------------+-------------+
 id_adesao| id_cancelado| id_vendedor|id_portal| id_loja|id_dt_emissao|id_dt_canc|valor_premio|valor_produto|
+--------------------+--------------------+------------+---------+--------------------+-------------+----------+------------+-------------+
48920010020606595701|48920010020606595701|489200000272| 4892|48920000240736940...| 48011| 48376| 191.88| 1599.0|
48920010020606611601|48920010020606611601|489200000403| 4892|48920000240736940...| 48016| 48017| 95.88| 799.0|
48920010020606632701|48920010020606632701|489200000425| 4892|48920000240736940...| 48025| 48025| 999.75| 3999.0|
48920010020606634301| null|489200000425| 4892|48920000240736940...| 48025| null| 999.75| 3999.0|
48920010020606638101| null|489200000205| 4892|48920000240736940...| 48026| null| 209.79| 999.0|
+--------------------+--------------------+------------+---------+--------------------+-------------+----------+------------+-------------+
only showing top 5 rows

In [6]:
fato.printSchema()

root
-- id_adesao: decimal(30,0) (nullable = true)
-- id_cancelado: decimal(30,0) (nullable = true)
-- id_vendedor: long (nullable = true)
-- id_portal: integer (nullable = true)
-- id_loja: decimal(30,0) (nullable = true)
-- id_dt_emissao: integer (nullable = true)
-- id_dt_canc: integer (nullable = true)
-- valor_premio: double (nullable = true)
-- valor_produto: double (nullable = true)

In [7]:
dim_adesao = spark.read.csv('/FileStore/tables/dim_adesao.csv',inferSchema=True,header=True)

dim_adesao.head(1)

Out[81]: [Row(id_adesao=Decimal('48920010020606595701'), cpf_cnpj=4560755426, nome_segurado='ALBERES COSTA E SILVA', sexo='Masculino', endereco='RUA ESQUILO', numero='310', bairro='OURO PRETO', cidade='OLINDA', UF='PE', nome_aparelho='GALAXY J7 PRIME 32GB DOURADO', categoria='SMARTPHONE', seguro='GARANTIA ESTENDIDA', desc_servico='GARANTIA ESTENDIDA ORIGINAL DO FORNECEDOR - SMARTPHONE', apolice=21959900000050, IMEI=359104086728646, data_emissao='2017-10-05', seguradora='SURA')]

###Joins with DataFrame

In [9]:
fato.join(dim_adesao, on=[fato.id_adesao == dim_adesao.id_adesao], how='right').head(1)

Out[91]: [Row(id_adesao=Decimal('48920010020606595701'), id_cancelado=Decimal('48920010020606595701'), id_vendedor=489200000272, id_portal=4892, id_loja=Decimal('4892000024073694004223'), id_dt_emissao=48011, id_dt_canc=48376, valor_premio=191.88, valor_produto=1599.0, id_adesao=Decimal('48920010020606595701'), cpf_cnpj=4560755426, nome_segurado='ALBERES COSTA E SILVA', sexo='Masculino', endereco='RUA ESQUILO', numero='310', bairro='OURO PRETO', cidade='OLINDA', UF='PE', nome_aparelho='GALAXY J7 PRIME 32GB DOURADO', categoria='SMARTPHONE', seguro='GARANTIA ESTENDIDA', desc_servico='GARANTIA ESTENDIDA ORIGINAL DO FORNECEDOR - SMARTPHONE', apolice=21959900000050, IMEI=359104086728646, data_emissao='2017-10-05', seguradora='SURA')]

In [10]:
for x in range(1,3):
  print("Register number: %i" % x )
  print(fato.join(dim_adesao, on='id_adesao', how='inner').filter(fato.id_cancelado.isNull()).head(x))
  print('---'*30)

Register number: 1
[Row(id_adesao=Decimal('48920010020606634301'), id_cancelado=None, id_vendedor=489200000425, id_portal=4892, id_loja=Decimal('4892000024073694004223'), id_dt_emissao=48025, id_dt_canc=None, valor_premio=999.75, valor_produto=3999.0, cpf_cnpj=10039452441, nome_segurado='jhony lucas guimarães', sexo='Masculino', endereco='PROFESSOR DIONÍSIO MACIEL MONTEIRO', numero='268', bairro='PEIXINHOS', cidade='OLINDA', UF='PE', nome_aparelho='GALAXY S8 64GB PRATA', categoria='SMARTPHONE', seguro='COMBO', desc_servico='ROUBO E FURTO COM QUEBRA ACIDENTAL DE PORTÁTEIS - SMARTPHONE', apolice=27109900000097, IMEI=358670081057596, data_emissao='2017-10-19', seguradora='SURA')]
------------------------------------------------------------------------------------------
Register number: 2
[Row(id_adesao=Decimal('48920010020606634301'), id_cancelado=None, id_vendedor=489200000425, id_portal=4892, id_loja=Decimal('4892000024073694004223'), id_dt_emissao=48025, id_dt_canc=None, valor_premio=999.75, valor_produto=3999.0, cpf_cnpj=10039452441, nome_segurado='jhony lucas guimarães', sexo='Masculino', endereco='PROFESSOR DIONÍSIO MACIEL MONTEIRO', numero='268', bairro='PEIXINHOS', cidade='OLINDA', UF='PE', nome_aparelho='GALAXY S8 64GB PRATA', categoria='SMARTPHONE', seguro='COMBO', desc_servico='ROUBO E FURTO COM QUEBRA ACIDENTAL DE PORTÁTEIS - SMARTPHONE', apolice=27109900000097, IMEI=358670081057596, data_emissao='2017-10-19', seguradora='SURA'), Row(id_adesao=Decimal('48920010020606638101'), id_cancelado=None, id_vendedor=489200000205, id_portal=4892, id_loja=Decimal('4892000024073694004223'), id_dt_emissao=48026, id_dt_canc=None, valor_premio=209.79, valor_produto=999.0, cpf_cnpj=934929440, nome_segurado='LEONARDO PRIMO DE CARVALHO', sexo='Masculino', endereco='GOVERNADOR AGAMENON MAGALHÃES', numero='4261', bairro='BOA VISTA', cidade='RECIFE', UF='PE', nome_aparelho='GALAXY J7 NEO 16GB PRETO', categoria='SMARTPHONE', seguro='ROUBO E FURTO', desc_servico='ROUBO E FURTO - SMARTPHONE', apolice=27109900000098, IMEI=359970080066658, data_emissao='2017-10-20', seguradora='SURA')]
------------------------------------------------------------------------------------------

In [11]:
dim_cancelado = spark.read.csv('/FileStore/tables/dim_cancelado.csv',inferSchema=True,header=True)

dim_cancelado.head(1)

Out[84]: [Row(id_cancelado=Decimal('48920010020606595701'), data_cancelamento='2018-10-05')]

In [12]:
fato.join(dim_cancelado, on='id_cancelado', how='left').select(fato.id_adesao.alias('fato.id_adesao'),fato.id_cancelado.alias('fato.id_cancelado'),
                                                               dim_cancelado.data_cancelamento.alias('dim_cancelado.dt_canc')).show(5)

+--------------------+--------------------+---------------------+
 fato.id_adesao| fato.id_cancelado|dim_cancelado.dt_canc|
+--------------------+--------------------+---------------------+
48920010020606595701|48920010020606595701| 2018-10-05|
48920010020606611601|48920010020606611601| 2017-10-11|
48920010020606632701|48920010020606632701| 2017-10-19|
48920010020606634301| null| null|
48920010020606638101| null| null|
+--------------------+--------------------+---------------------+
only showing top 5 rows

###SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [14]:
# Register the DataFrame as a SQL temporary view
fato.createOrReplaceTempView("fato")
dim_adesao.createOrReplaceTempView('dim_adesao')
dim_cancelado.createOrReplaceTempView('dim_cancelado')

In [15]:
spark.sql('''SELECT d.nome_segurado, a.id_adesao, d.nome_aparelho,a.valor_premio, c.id_cancelado  
              FROM fato a
              JOIN dim_adesao d
              ON d.id_adesao = a.id_adesao
              LEFT JOIN dim_cancelado c
              ON c.id_cancelado = a.id_cancelado
              WHERE a.valor_premio > 500''').show()

+--------------------+--------------------+--------------------+------------+--------------------+
 nome_segurado| id_adesao| nome_aparelho|valor_premio| id_cancelado|
+--------------------+--------------------+--------------------+------------+--------------------+
jhony lucas guim...|48920010020606632701|GALAXY S8 64GB PRATA| 999.75|48920010020606632701|
jhony lucas guim...|48920010020606634301|GALAXY S8 64GB PRATA| 999.75| null|
SUAMI VAN DER LIN...|48920010020606659801|GALAXY NOTE 8 128...| 1007.79| null|
SANDRO HENRIQUE D...|48920010020606664001|GALAXY NOTE 8 128...| 881.79| null|
GIOVANNI BISPO MATOS|48920010020606670601|GALAXY A7 2017 64...| 574.75| null|
 DANNYLLO MOTA|48920010020606748901|GALAXY S8 64GB AM...| 999.75| null|
ERICA TAIRINE DE ...|48920010020606750201|GALAXY S8 PLUS 64...| 1099.75| null|
CONSTRUTORA E IMC...|48920010020606781701|GALAXY S8 PLUS 64...| 703.84| null|
+--------------------+--------------------+--------------------+------------+--------------------+